In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
import time
import sys

import requests
from bs4 import BeautifulSoup

#Set options according to the browser you use
# Get all info from the link below
#https://www.selenium.dev/documentation/en/getting_started_with_webdriver/browsers/
options = webdriver.FirefoxOptions()

#uncomment the below code if you don't want a browser window to open
# options.add_argument('--headless') 

flag = 0
cchc = ''

#will contain the peices of all websites and show the best price accordingly
final_price_list = []

def inputData():
    proItem = input("Enter product name:\n")
    itemName = proItem.replace(" ","+")
    
    print("\n\nWebsites you can compare your product from\n[1] Flipkart\n[2] Amazon\n[3] Croma\n[4] Ajio \n\n More websites coming soon! \n\n")
    cchc == input("Start Comparing? [Y/N]:\n")
        
    if cchc == 'Y' or 'y':
        print("\n\n\nCool let's start then!")
        flipkart(itemName)
    else:
        exit(1)
        

#Flipkart
def flipkart(itemName):
    print("Let's start scraping flipkart......")
    itemName = itemName
    flipurl = f'https://www.flipkart.com/search?q={itemName}&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off'
    headers = {
    'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0'
    }

    #scrape the main pro_list page of item
    req = requests.get(flipurl,headers=headers)
    soup = BeautifulSoup(req.content,'lxml')

    try:

        #main list
        pro_list = soup.find_all('div',class_='_2kHMtA')

        #will contain links of all product in the list
        proLink = []

        for i in pro_list:
            for link in i.find_all('a',href=True):
                proLink.append(flipurl + link['href'])


        #scrape data from particular item
        itemLink = proLink[0]

        req = requests.get(itemLink,headers=headers)
        soup = BeautifulSoup(req.content,'lxml')
        image = soup.find_all('div',class_="CXW8mj _3nMexc")

        for name in soup.find_all('span',class_="B_NuCI"):
            pro_name = name.text.strip()
        for price in soup.find_all('div',class_="_30jeq3 _16Jk6d"):
            pro_price = price.text.strip()
        for img in image:
            pro_image = img.get('src')

        #append the price scraped in float form
        temp_pro_price = pro_price.replace("₹","")
        final_pro_price = float(temp_pro_price.replace(",",""))
        final_price_list.append(final_pro_price)
        
        print("\n")    
        print(itemLink)
        print(pro_name)
        print(pro_price)
        print(type(final_pro_price))
        print(pro_image)
        print("\n\n\nNow Let's scrape Amazon.....\n")
        amazon(itemName)
    ################

    except:
        #main list
        pro_list = soup.find_all('div',class_='_1xHGtK _373qXS')

        #will contain links of all product in the list
        proLink = []

        for i in pro_list:
            for link in i.find_all('a',href=True):
                proLink.append(flipurl + link['href'])


        #scrape data from particular item
        itemLink = proLink[0]

        req = requests.get(itemLink,headers=headers)
        soup = BeautifulSoup(req.content,'lxml')
        image = soup.find_all('div',class_="CXW8mj _3nMexc")

        for name in soup.find_all('span',class_="G6XhRU"):
            pro_name = name.text.strip()
        for price in soup.find_all('div',class_="_30jeq3 _16Jk6d"):
            pro_price = price.text.strip()
    #     for img in image:
    #         pro_image = img.get('src')

        print("\n")    
        print(itemLink)
        print(pro_name)
        print(pro_price)
    #     print(pro_image)
        print("\n\n\nNow Let's scrape Amazon.....\n")
        amazon(itemName)
    

#amazon
def amazon(itemName):
    itemName = itemName
    amazon_url = f'https://www.amazon.in/s?k={itemName}&ref=nb_sb_noss'
    headers = {
        'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0'
    }

    #scrape the main pro_list page of item
    req = requests.get(amazon_url,headers=headers)
    soup = BeautifulSoup(req.content,'lxml')

    #list of all the products on the newUrl
    pro_list = soup.find_all('div',{'data-component-type':'s-search-result'})
    try:
        prod = pro_list[0]

        imageTag = prod.find('div',class_='a-section aok-relative s-image-fixed-height')
        prodImage = imageTag.find('img')
        prodImageLink = prodImage['src']
        print("Image Link:",prodImageLink)

        atag = prod.h2.a
        prodName = atag.text

        # prodUrl = amazon_url + atag.find('a',href=True)

        #prod price
        price = prod.find('span',class_="a-price")
        amazonPrice = price.find('span',class_="a-offscreen").text
        
        #append the price scraped in float form
        temp_pro_price = amazonPrice.replace("₹","")
        final_pro_price = float(temp_pro_price.replace(",",""))
        final_price_list.append(final_pro_price)

        print("\nName:",prodName)
        # print("URL:\n",prodUrl)
        print("Product Link: ",amazon_url)
        print("Price:",amazonPrice)
        time.sleep(2)
        print("\n\n\nNow Let's scrape Croma.........\n")
        croma(itemName)
    except:
        print("Product not found!")
        print("\n\n\nNow Let's scrape Croma.........\n")
        croma(itemName)
        
        
        
def croma(itemName):
    itemName = itemName
    croma_base_url = 'https://www.croma.com'
    croma_url = f'https://www.croma.com/search/?text={itemName}'
    driver = webdriver.Firefox(executable_path="D:\\geckodriver-v0.28.0-win64\\geckodriver.exe",options=options)
    headers = {
        'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0'
    }

    driver.get(croma_url)

    soup=BeautifulSoup(driver.page_source,'html.parser')

    #list of all the products on the newUrl
    pro_list = []
    pro = soup.findAll('div',class_='content-wrap')
    for j in pro:
        for i in j.findAll('ul',class_='product-list'):
    #         print(i)
            for k in i.findAll('li',class_='product-item'):
                pro_list.append(k)

    try:
        prod = pro_list[0]

        imageTag = prod.find('div',class_='product-img')
        prodImage = imageTag.find('img')
        prodImageLink = prodImage['src']
        print("Image Link:",prodImageLink)

        atag = prod.h3.a
        prodLink = croma_base_url + atag['href']
        print("\nProduct Link:",prodLink)

        name = prod.h3.text
        print("Name:",name)

        cromaPrice = prod.find('span',class_='amount').text
        
        #append the price scraped in float form
        temp_pro_price = cromaPrice.replace("₹","")
        final_pro_price = float(temp_pro_price.replace(",",""))
        final_price_list.append(final_pro_price)
        
        print("\nPrice:",cromaPrice)
        print("\n\n\nNow Let's scrape Ajio.........\n")
        ajio(itemName)
    except:
        print("Product not found!")
        print("\n\n\nNow Let's scrape Ajio.........\n")
        ajio(itemName)

        
        
def ajio(itemName):
    itemName = itemName
    ajio_base_url = 'https://www.ajio.com/'
    ajio_url = f'https://www.ajio.com/search/?text={itemName}'
    driver = webdriver.Firefox(executable_path="D:\\geckodriver-v0.28.0-win64\\geckodriver.exe",options=options)
    headers = {
        'User-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:83.0) Gecko/20100101 Firefox/83.0'
    }

    driver.get(ajio_url)
    time.sleep(2)

    soup=BeautifulSoup(driver.page_source,'html.parser')

    pro_list = []
    pro = soup.findAll('div',class_='ReactVirtualized__Grid items')
    for j in pro:
        for i in j.findAll('div',class_='ReactVirtualized__Grid__innerScrollContainer'):
    #         print(i)
            for k in i.find('div',class_='item rilrtl-products-list__item item'):
                pro_list.append(k)

    try:
        prod = pro_list[0]

        imageTag = prod.find('div',class_='imgHolder')
        prodImage = imageTag.find('img')
        prodImageLink = prodImage['src']
        print(prodImageLink)

        name = prod.find('div',class_='name')
        ajioName = name.text
        print("Name:",ajioName)

        ajioPrice = prod.find('span',class_='price').text
        
        #append the price scraped in float form
        temp_pro_price = ajioPrice.replace("₹","")
        final_pro_price = float(temp_pro_price.replace(",",""))
        final_price_list.append(final_pro_price)
        
        print("\nPrice:",ajioPrice)   
        price = ajioPrice
        bestBuy(itemName)
    except:
        print("Product Not Found!")   
        bestBuy(itemName)

        


    
def bestBuy(itemName):
    itemName = itemName
    print("\n\n")
    print("The best price for the %s is:\t"%(itemName.replace('+','')))
    print(min(final_price_list))
    print("\n\n")
    
    
while flag != 1:
    flag = int(input("Enter [0] to continue and [1] to exit:\n"))
    if flag == 0:
        inputData()
        
    else:
        exit(1)
        
    flag = int(input("Enter [0] to continue and [1] to exit:\n"))

Enter [0] to continue and [1] to exit:
0
Enter product name:
iphone 11


Websites you can compare your product from
[1] Flipkart
[2] Amazon
[3] Croma
[4] Ajio 

 More websites coming soon! 


Start Comparing? [Y/N]:
Y



Cool let's start then!
Let's start scraping flipkart......


https://www.flipkart.com/search?q=iphone+11&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off/apple-iphone-11-black-64-gb/p/itm4e5041ba101fd?pid=MOBFWQ6BXGJCEYNY&lid=LSTMOBFWQ6BXGJCEYNYZXSHRJ&marketplace=FLIPKART&q=iphone+11&store=tyy%2F4io&srno=s_1_1&otracker=search&otracker1=search&fm=organic&iid=19db0474-b3d0-4b26-87b1-0c88de5f7e21.MOBFWQ6BXGJCEYNY.SEARCH&ppt=None&ppn=None&ssid=3lg0oss7xc0000001627166628758&qH=f6cdfdaa9f3c23f3
APPLE iPhone 11 (Black, 64 GB)
₹49,999
<class 'float'>
None



Now Let's scrape Amazon.....

Image Link: https://m.media-amazon.com/images/I/71i2XhHU3pL._AC_UY218_.jpg

Name: New Apple iPhone 11 (64GB) - Black 
Product Link:  https://www.amazon.in/s?k=iphone+11&

IndexError: list index out of range